In [ ]:
!nvidia-smi

Sun Jul  2 14:08:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install split-folders

In [ ]:
import splitfolders

input_folder="/content/drive/MyDrive/Dataset/dataset2"


output="/content/"


splitfolders.ratio(input_folder, output, seed=42, ratio=(.80, .10, .10))

In [ ]:
### model

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow import keras
import numpy as np
from glob import glob
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

IMAGE_SIZE = [224,224]

######################################### give class number ###########
CLASS=4

inception = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#model = ResNet50(weights='imagenet', include_top=False)

for layer in inception.layers:
    layer.trainable = False

#folders = glob('C:\rafid\guava disease research\k_guava\train_image/*')

x = Flatten()(inception.output)

prediction = Dense(CLASS, activation='softmax')(x)

model = Model(inputs=inception.input, outputs=prediction)
adam = keras.optimizers.Adam(lr=0.001)
model.compile(
  loss='categorical_crossentropy',
  optimizer=adam,
  metrics=['accuracy']
)




model.optimizer.get_config()
print("\n\n")
model.summary()

In [ ]:
# sayma comment out line 2, Ritu comment out line 1 before running

op = keras.optimizers.Adam(lr=0.001)  #for Sayma
#op = keras.optimizers.Nadam(lr=0.0001)  # for Ritu
model.compile(
  loss='categorical_crossentropy',
  optimizer=op,
  metrics=['accuracy']
)

In [ ]:



train_datagen = ImageDataGenerator(rescale = 1./255,
                                  # shear_range = 0.2,
                                  # zoom_range = 0.2,
                                   brightness_range=(0.4, 0.7),
                                   vertical_flip= True,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(r'/content/train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

val_set = val_datagen.flow_from_directory(r'/content/val',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(r'/content/test',
                                           target_size = (224,224),
                                           batch_size = 1,
                                           class_mode = 'categorical')



model.optimizer.get_config()

In [ ]:
from PIL import Image

filepath = r"/content/drive/MyDrive/Dataset/mobilenet/vgg16_s80.h5"

### example___filepath = "path to weight directory/breastCancer_geometric.h5"


checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_weights_only=True,
                             save_best_only=True, mode='max')

log_csv = CSVLogger(r'/content/drive/MyDrive/Dataset/mobilenet/VGG16_s80.csv', separator=',', append=False)
### example___CSVLogger('path to logs directory/breastCancer_geometric.csv')

callbacks_list = [checkpoint1,log_csv]


r = model.fit_generator(
    training_set,
    epochs=160,
    validation_data=val_set,
    steps_per_epoch = len(training_set),
    validation_steps=len(val_set),
    callbacks=callbacks_list,
    shuffle=False


)



In [ ]:
model.load_weights(r"/content/drive/MyDrive/Dataset/mobilenet/vgg16_s80.h5")
Adam = keras.optimizers.Adam(lr=0.001)


preds = model.evaluate_generator(val_set)
print ("Loss = ",float(preds[0]))
print ("Test Accuracy = ",float(preds[1])*100)

<ipython-input-13-085658f506aa>:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  preds = model.evaluate_generator(val_set)


Loss =  4.617265224456787
Test Accuracy =  92.76595711708069


In [ ]:
model.load_weights(r"/content/drive/MyDrive/Dataset/mobilenet/vgg16_s80.h5")
Adam = keras.optimizers.Adam(lr=0.001)


preds = model.evaluate_generator(test_set)
print ("Loss = ",float(preds[0]))
print ("Test Accuracy = ",float(preds[1])*100)

<ipython-input-14-ae91665d95db>:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  preds = model.evaluate_generator(test_set)


Loss =  4.8968377113342285
Test Accuracy =  89.64059352874756


In [ ]:
### loss and accuracy curve ### updated !!!!!!!!!!

import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt


fin1= pd.read_csv(r'/content/drive/MyDrive/Dataset/mobilenet/VGG16_s80.csv') ###directory


### accuracy curve ###
plt.figure(figsize=(7,4))
plt.plot(fin1['accuracy'], label='Trining accuracy')
plt.plot(fin1['val_accuracy'], label='Validation accuracy')

plt.title('Accuracy Curve')
plt.legend(loc='lower right')

plt.savefig(r'D:\Inam Ullah Khan\RestNet50\Zrms.png', dpi = 300)

### loss curve ###

plt.figure(figsize=(7,4))
plt.plot(fin1['loss'], label='Trining loss')
plt.plot(fin1['val_loss'], label='Validation loss')

plt.title('Loss Curve')

plt.savefig(r'D:\Inam Ullah Khan\RestNet50\Zrms.png', dpi = 300)
plt.legend(loc='upper right')

In [ ]:
### for f1 score and other values import files ###

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
#importing packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
### matrics
from sklearn import metrics
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
####predictions on test set

y_test=[]
probaa=[]
import os
import matplotlib.pyplot as plt
model.load_weights(r"/content/drive/MyDrive/Dataset/mobilenet/vgg16_s80.h5")


cls=4  #nummber of classes

## give directory of class serially..if less than 4 classes comment out the last unused dir_path
dir_path1=r'/content/val/Botrytis Blight'
dir_path2=r'/content/val/Downy Mildew'
dir_path3=r'/content/val/Purple Blotch'
dir_path4=r'/content/val/Stemphylium leaf blight'



w1=0
w2=0
w3=0
w4=0

if cls==5:
  for filename in os.listdir(dir_path1):
      img=image.load_img(dir_path1+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(0)
      w1+=1
      print(w1,end='_')

  print('\n')
  for filename in os.listdir(dir_path2):
      img=image.load_img(dir_path2+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(1)
      #if res!=1: a2+=1
      w2+=1
      print(w2,end='_')

  print('\n')
  for filename in os.listdir(dir_path3):
      img=image.load_img(dir_path3+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(2)
      w3+=1
      print(w3,end='_')

  print('\n')
  for filename in os.listdir(dir_path4):
      img=image.load_img(dir_path4+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(3)
      w4+=1
      print(w4,end='_')

  for filename in os.listdir(dir_path5):
      img=image.load_img(dir_path4+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(3)
      w4+=1
      print(w4,end='_')
  print('class numbers', cls)
  print('prediction number:',len(probaa))
  print('true labels:',len(y_test))

if cls==4:
  for filename in os.listdir(dir_path1):
      img=image.load_img(dir_path1+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(0)
      w1+=1
      print(w1,end='_')

  print('\n')
  for filename in os.listdir(dir_path2):
      img=image.load_img(dir_path2+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(1)
      #if res!=1: a2+=1
      w2+=1
      print(w2,end='_')

  print('\n')
  for filename in os.listdir(dir_path3):
      img=image.load_img(dir_path3+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(2)
      w3+=1
      print(w3,end='_')

  print('\n')
  for filename in os.listdir(dir_path4):
      img=image.load_img(dir_path4+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(3)
      w4+=1
      print(w4,end='_')

  print('class numbers', cls)
  print('prediction number:',len(probaa))
  print('true labels:',len(y_test))


if cls==2:
  for filename in os.listdir(dir_path1):
      img=image.load_img(dir_path1+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(0)
      w1+=1
      print(w1,end='_')

  print('\n')
  for filename in os.listdir(dir_path2):
      img=image.load_img(dir_path2+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(1)
      #if res!=1: a2+=1
      w2+=1
      print(w2,end='_')

  print('\n')

  print('class numbers', cls)
  print('prediction number:',len(probaa))
  print('true labels:',len(y_test))


if cls==3:
  for filename in os.listdir(dir_path1):
      img=image.load_img(dir_path1+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(0)
      w1+=1
      print(w1,end='_')

  print('\n')
  for filename in os.listdir(dir_path2):
      img=image.load_img(dir_path2+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(1)
      w2+=1
      print(w2,end='_')

  print('\n')
  for filename in os.listdir(dir_path3):
      img=image.load_img(dir_path3+'//'+filename, target_size=(224,224,3))
      im = []
      img = image.img_to_array(img)
      img = img/255
      im.append(img)
      X= np.array(im)
      result=model.predict(X,verbose=0)
      #training_set.class_indices
      res=np.argmax(result)
      probaa.append(res)
      y_test.append(2)
      w3+=1
      print(w3,end='_')

  print('\n')

  print('class numbers', cls)
  print('prediction number:',len(probaa))
  print('true labels:',len(y_test))

In [ ]:
#We use Support Vector classifier as a classifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
#importing packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
### matrics
from sklearn import metrics
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
##  make a directory for storing figures
y_preD=probaa
cm = confusion_matrix(y_test, y_preD)
cm_df = pd.DataFrame(cm,
                     index = ['Botrytis Blight','Downy Mildew','Purple Blotch','Stemphylium leaf blight'],
                     columns = ['Botrytis Blight','Downy Mildew','Purple Blotch','Stemphylium leaf blight'])

#Plotting the confusion matrix

#import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
sns.heatmap(cm_df,cmap='cividis', annot=True, fmt='d')
#plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')

###save confusion matrix to fig directory ...keep the image file name as h5 file name. ex:  breastCancer_geometric.h5 >>> breastCancer_geometric_con.png
plt.savefig("/content/drive/MyDrive/Dataset/mobilenet\_s-(80)_con.png", dpi = 300)
plt.show()


In [ ]:
y_preD=probaa
y_val=y_test

confusion_matrix=cm

FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = len(y_val) - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP)
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math
from numpy import mean

MAE=mean_absolute_error(y_val, y_preD)
MSE=mean_squared_error(y_val, y_preD)
F1=2*((mean(PPV)*mean(TPR))/(mean(PPV)+mean(TPR)))

#print("ACC=   ",mean(ACC)*100)

print("Rec=   ",mean(TPR)*100)
print("Spe=   ",mean(TNR)*100)
print("Pre=   ",mean(PPV)*100)
print("FPR=   ",mean(FPR)*100)
print("FNR=   ",mean(FNR)*100)
print("NPV=   ",mean(NPV)*100)
print("FDR=   ",mean(FDR)*100)
print("F1=    ",mean(F1)*100)

print("MAE   =",MAE*100)
print("RMSE  =",math.sqrt(MSE)*100)


